Imagine we are in a Steel setup

I  want to predict equipment failure
➡️ Binary classification

1 = Failure soon

0 = Normal operation

I  have sensor data:

Temperature

Pressure

Vibration

Load

But one constraint

Dataset is small

Features are noisy

Model overfits easily

Wrong predictions are expensive

So I don’t just want accuracy —
I want robust + calibrated + uncertainty-aware predictions.

That’s why i will bring

ANN

SAM

Label Smoothing

Monte Carlo Dropout

ANN (Base model)

Learns non-linear relationships
But
❌ Can overfit
❌ Overconfident predictions

Label Smoothing

Instead of saying:

Failure = 1.0


We say:

Failure = 0.9
Not failure = 0.1

Reduces overconfidence

Improves generalization

It Makes probabilities more realistic

MY USP



🧗 SAM (Sharpness-Aware Minimization)

Avoids sharp minima

Finds flat, generalizable solutions

Massive overfitting killer

🎲 Monte Carlo Dropout

Dropout during inference, not just training.

Run model N times, get distribution of predictions.


Gives uncertainty

Helps risk-based decisions

Crucial in industrial AI


Model says 0.92 ± 0.15

mean plus minus variance

Dataset (Synthetic but realistic)

I simulate sensor data

i am using PyTorch for model definition and optimization, and sklearn for data preprocessing to ensure numerical stability

In [42]:
#Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [43]:
#Fixing Randomness
torch.manual_seed(42)
np.random.seed(42)

In [44]:
#My Synthetic Dataset Creation
n_samples = 300

temperature = np.random.normal(70, 10, n_samples) # mean std
pressure = np.random.normal(30, 5, n_samples)
vibration = np.random.normal(0.5, 0.2, n_samples)
load = np.random.normal(80, 15, n_samples)

X = np.vstack([temperature, pressure, vibration, load]).T

In [45]:
y = (
    (temperature > 80).astype(int) |
    (vibration > 0.8).astype(int) |
    ((pressure > 35) & (load > 90)).astype(int)
)


#Creates non-linear failure conditions.



# Failure is not linear

# Multiple interacting causes

# Perfect use case for ANN

Failure labels are generated  by me via non-linear thresholds to reflect real operational constraints.

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

ANNs assume:

features centered around 0

similar scales

Otherwise gradients go beserk

I  have standardized inputs to improve gradient flow and training stability.

In [47]:
#Converting to PyTorch Tensors from Numpy

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

input → hidden → hidden → output

dropout regularization

2 hidden layers with droput

In [48]:
class FailureANN(nn.Module):
    """
    Feedforward ANN for equipment failure prediction.

    Architecture:
    - Input: 4 sensor features
    - Hidden layer 1: 32 neurons + ReLU + Dropout
    - Hidden layer 2: 16 neurons + ReLU + Dropout
    - Output: 2 logits (binary classification)
    """

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        """
        Forward pass.

        Dropout serves two roles:
        - Regularization during training
        - Uncertainty sampling during MC Dropout
        """
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)

In [49]:
# MODEL INSTANTIATION

model = FailureANN()

ReLU adds non-linearity

Dropout adds  robustness by randomly dropping some neurons

Also Non-linear activations allow the network to model complex feature interactions

My USP Label Smoothing

In [50]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

Label smoothing Softens target labels

it exists and

Prevents:

extreme confidence

brittle decision boundaries

Label smoothing improves calibration and reduces overconfidence


**SHARPNESS AWARE MINIMIZATION**

In [51]:
#SAM Optimizer
#First step


#optimizer.first_step()

#This Moves weights toward worst-case sharp direction


In [52]:
#Second step
#optimizer.second_step()

#Optimizes loss in that perturbed neighborhood

SAM exists

Flat minima generalize

Sharp minima overfit

This is also known as MIN MAX optimizzation

# PART 1️⃣ —Susnata SAM (Sharpness-Aware Minimization)
1. What problem SAM is solving

Standard optimizers (SGD, Adam) do this:

“Find parameters θ that minimize training loss.”

SAM does something stricter:

“Find parameters θ whose neighborhood also has low loss.”

Why so if one asks ?

Sharp minima → overfit → break on new data

Flat minima → robust → generalize

I put this mentak model Think terrain:

Sharp minimum = needle peak

Flat minimum = wide valley

SAM wants the valley 🏞️

# I  slightly push the weights in the direction that increases loss the most.

#Susnata SAM Step 2 — Optimize for flatness

Now I:

pull weights back

update parameters to minimize the worst-case loss

So the optimizer learns parameters that:

are good at θ

are still good at θ + ε

My intuition

“I only accept solutions that are stable under small perturbations.”

That’s why SAM generalizes better.

SAM tests the local minimum once — but it tests the worst possible direction.

#Forward pass

#Loss computation

#SAM step 1

# SAM step 2

In [53]:
model = FailureANN()

In [54]:
class SAM(torch.optim.Optimizer):
    """
    Sharpness-Aware Minimization (SAM) Optimizer.

    SAM improves generalization by optimizing for parameters that lie in
    flat regions of the loss landscape rather than sharp minima.

    Instead of minimizing only the training loss L(θ),
    SAM minimizes the worst-case loss in a small neighborhood around θ.

    This is a MIN-MAX optimization:
        min_θ max_{||ε|| ≤ ρ} L(θ + ε)

    Parameters
    ----------
    params : iterable
        Model parameters to optimize.

    base_optimizer : torch.optim.Optimizer
        The inner optimizer (e.g., Adam, SGD).

    rho : float, default=0.05
        Radius of the neighborhood used to test sharpness.
        Larger rho = stronger flatness constraint.

    **kwargs :
        Additional arguments passed to the base optimizer.
    """

    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        # Initialize the base optimizer (Adam / SGD)
        self.base_optimizer = base_optimizer(params, **kwargs)

        # SAM shares the same parameter groups as the base optimizer
        self.param_groups = self.base_optimizer.param_groups

        # Neighborhood size for sharpness testing
        self.rho = rho

        # Store perturbations for each parameter
        self.state = {}

    @torch.no_grad()
    def first_step(self):
        """
        First SAM step: Sharpness probing step.

        This step perturbs the model parameters in the direction
        that maximally increases the loss (worst-case direction).

        Intuition:
        - Find how sharp the current minimum is
        - Push parameters slightly toward higher loss
        """

        # Compute L2 norm of all gradients
        grad_norm = torch.norm(
            torch.stack([
                p.grad.norm(p=2)
                for group in self.param_groups
                for p in group['params']
                if p.grad is not None
            ])
        )

        # Apply perturbation to each parameter
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue

                # Compute scaled perturbation
                eps = self.rho * p.grad / (grad_norm + 1e-12)

                # Move parameter to worst-case position
                p.add_(eps)

                # Store perturbation for rollback
                self.state[p] = {"eps": eps}

    @torch.no_grad()
    def second_step(self):
        """
        Second SAM step: Flatness-aware optimization step.

        This step:
        - Rolls back the perturbation
        - Updates parameters to minimize the worst-case loss

        Intuition:
        - Only accept parameter updates that are stable
          under small perturbations
        """

        # Roll back perturbations
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                p.sub_(self.state[p]["eps"])

        # Perform the actual optimizer update
        self.base_optimizer.step()

    def zero_grad(self):
        """
        Clears gradients of all optimized parameters.
        """
        self.base_optimizer.zero_grad()


In [55]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = SAM(
    model.parameters(),
    base_optimizer=Adam,
    lr=0.001
)

In [56]:
epochs = 40

for epoch in range(epochs):
    model.train()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # SAM step 1
    loss.backward()
    optimizer.first_step()

    # SAM step 2
    optimizer.zero_grad()
    criterion(model(X_train), y_train).backward()
    optimizer.second_step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")


Epoch 0 | Loss: 0.6695
Epoch 10 | Loss: 0.6506
Epoch 20 | Loss: 0.6368
Epoch 30 | Loss: 0.6143


In [57]:
def mc_dropout_predict(model, X, n_runs=50):
    """
    Perform Monte Carlo Dropout inference to estimate prediction uncertainty.

    Parameters
    ----------
    model : torch.nn.Module
        A trained PyTorch neural network containing Dropout layers.
        The model weights are assumed to be frozen (no further training).

    X : torch.Tensor
        Input feature tensor of shape (n_samples, n_features).

    n_runs : int, default=50
        Number of stochastic forward passes to perform.
        Higher values give smoother uncertainty estimates but increase computation.

    Returns
    -------
    mean_probs : np.ndarray
        Mean predicted class probabilities across Monte Carlo runs.
        Shape: (n_samples, n_classes)

    std_probs : np.ndarray
        Standard deviation of predicted probabilities across runs.
        Represents epistemic uncertainty.
        Shape: (n_samples, n_classes)
    """

    # Enable dropout layers during inference.
    # This does NOT train the model or update weights.
    model.train()# Enable dropout (no training)

    # Store probability predictions from each stochastic forward pass
    probs = []

    for _ in range(n_runs):
        # Forward pass only (no gradient computation)
        logits = model(X)

        # Convert logits to probabilities and detach from computation graph
        prob = F.softmax(logits, dim=1).detach().cpu().numpy()

        # Collect prediction from this run
        probs.append(prob)

    # Convert list to array:
    # Shape -> (n_runs, n_samples, n_classes)
    probs = np.array(probs)

    # Mean across Monte Carlo runs (axis=0)
    # Represents final prediction
    mean_probs = probs.mean(axis=0)

    # Standard deviation across Monte Carlo runs (axis=0)
    # Represents model uncertainty
    std_probs = probs.std(axis=0)

    return mean_probs, std_probs

    #prob +- uncertainity

In [58]:
mean_probs, uncertainty = mc_dropout_predict(model, X_test)

print("Failure probability:", mean_probs[0][1])
print("Uncertainty:", uncertainty[0][1])

Failure probability: 0.37035656
Uncertainty: 0.03751995


In [62]:
failure_prob = mean_probs[:, 1]        # Class 1 = Failure
failure_uncertainty = uncertainty[:, 1]

In [63]:
import pandas as pd

df_predictions = pd.DataFrame({
    "failure_probability": failure_prob,
    "uncertainty": failure_uncertainty
})

In [64]:
df_predictions["predicted_label"] = (
    df_predictions["failure_probability"] > 0.5 # chosing a threshold of 0.5
).astype(int)


# 1 → predicted failure

# 0 → normal

In [65]:
def risk_bucket(row):
    if row["failure_probability"] > 0.8 and row["uncertainty"] < 0.1:
        return "HIGH RISK – ACT"
    elif row["failure_probability"] > 0.6:
        return "MEDIUM RISK – MONITOR"
    else:
        return "LOW RISK"

df_predictions["risk_level"] = df_predictions.apply(risk_bucket, axis=1)

In [66]:
df_predictions.head()

,failure_probability,uncertainty,predicted_label,risk_level
0,0.370357,0.037520,0,LOW RISK
1,0.387138,0.048560,0,LOW RISK
2,0.367189,0.031987,0,LOW RISK
3,0.436074,0.044208,0,LOW RISK
4,0.372602,0.036304,0,LOW RISK


In [67]:
df_sensors = pd.DataFrame(
    scaler.inverse_transform(X_test.numpy()),
    columns=["temperature", "pressure", "vibration", "load"]
)

df_final = pd.concat([df_sensors, df_predictions], axis=1)
df_final.head()

,temperature,pressure,vibration,load,failure_probability,uncertainty,predicted_label,risk_level
0,80.538017,32.814846,0.407545,77.906158,0.370357,0.037520,0,LOW RISK
1,86.324112,19.809378,0.557173,100.764091,0.387138,0.048560,0,LOW RISK
2,63.199753,25.071369,0.304825,87.410454,0.367189,0.031987,0,LOW RISK
3,75.425598,28.989037,0.799209,112.448822,0.436074,0.044208,0,LOW RISK
4,63.483997,23.944920,0.346440,86.803017,0.372602,0.036304,0,LOW RISK


In [68]:
# Adding uncertainty bounds
df_final["failure_prob_lower"] = (
    df_final["failure_probability"] - df_final["uncertainty"]
).clip(0, 1)

df_final["failure_prob_upper"] = (
    df_final["failure_probability"] + df_final["uncertainty"]
).clip(0, 1)

In [69]:
df_final.head()


,temperature,pressure,vibration,load,failure_probability,uncertainty,predicted_label,risk_level,failure_prob_lower,failure_prob_upper
0,80.538017,32.814846,0.407545,77.906158,0.370357,0.037520,0,LOW RISK,0.332837,0.407877
1,86.324112,19.809378,0.557173,100.764091,0.387138,0.048560,0,LOW RISK,0.338578,0.435699
2,63.199753,25.071369,0.304825,87.410454,0.367189,0.031987,0,LOW RISK,0.335202,0.399175
3,75.425598,28.989037,0.799209,112.448822,0.436074,0.044208,0,LOW RISK,0.391866,0.480281
4,63.483997,23.944920,0.346440,86.803017,0.372602,0.036304,0,LOW RISK,0.336298,0.408907


In [70]:
def risk_from_bounds(row):
    if row["failure_prob_upper"] >= 0.7:
        return "HIGH RISK"
    elif row["failure_prob_upper"] >= 0.4:
        return "MEDIUM RISK"
    else:
        return "LOW RISK"

df_final["risk_level_bounds"] = df_final.apply(risk_from_bounds, axis=1)

In [72]:
df_final.head()

,temperature,pressure,vibration,load,failure_probability,uncertainty,predicted_label,risk_level,failure_prob_lower,failure_prob_upper,risk_level_bounds
0,80.538017,32.814846,0.407545,77.906158,0.370357,0.037520,0,LOW RISK,0.332837,0.407877,MEDIUM RISK
1,86.324112,19.809378,0.557173,100.764091,0.387138,0.048560,0,LOW RISK,0.338578,0.435699,MEDIUM RISK
2,63.199753,25.071369,0.304825,87.410454,0.367189,0.031987,0,LOW RISK,0.335202,0.399175,LOW RISK
3,75.425598,28.989037,0.799209,112.448822,0.436074,0.044208,0,LOW RISK,0.391866,0.480281,MEDIUM RISK
4,63.483997,23.944920,0.346440,86.803017,0.372602,0.036304,0,LOW RISK,0.336298,0.408907,MEDIUM RISK


# I report failure probability along with uncertainty bounds, enabling risk-aware decisions instead of single-point predictions.

In [73]:
df_final.to_csv("equipment_failure_predictions.csv", index=False)

In [74]:
from google.colab import files
files.download("equipment_failure_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



The objective of this model is robust equipment failure prediction under conditions of:

small datasets

noisy sensor signals

correlated input features

high cost of wrong predictions

The focus is reliable decision-making, not per-feature attribution.

🔹 Handling Feature Correlation

The input features (temperature, pressure, vibration, load) exhibit natural correlation, which is common in industrial sensor data.

Instead of explicitly removing or decorrelating features, we allow the neural network to learn latent representations, where correlated inputs are absorbed into internal feature combinations.

This avoids:

coefficient instability (as in linear models)

forced feature importance separation

misleading attribution artifacts

🔹 Why Feature Attribution (SHAP) Was Not Used

Feature attribution methods such as SHAP assume:

conditional feature independence

stable attribution paths

In the presence of correlated inputs, SHAP:

arbitrarily splits importance

produces unstable explanations

creates false interpretability

Using SHAP in this setting would introduce explanation noise, not clarity.

Therefore, feature-level explainability was intentionally avoided.

🔹 Explainability Strategy Used Instead

The model emphasizes decision-level explainability, implemented through:

Calibration (Label Smoothing)
→ probabilities are realistic, not overconfident

Robust Optimization (SAM)
→ predictions remain stable under small perturbations

Uncertainty Quantification (Monte Carlo Dropout)
→ each prediction is accompanied by an uncertainty estimate

This allows outputs such as:

Failure Probability = 0.92 ± 0.15

🔹 When Feature Explainability Can Be Added

If in future explainability is required in future iterations, only the following approaches are appropriate:

Group-level SHAP
(cluster correlated features and explain groups)

Latent-factor explanations
(PCA → ANN → interpret hidden representations)

Scenario-based analysis
(“What happens if vibration increases by X under fixed conditions?”)

Naive per-feature SHAP on raw correlated inputs is explicitly avoided.